In [ ]:
ETUDE HYPERPARAMETRES TVAE TE CTGAN 

In [2]:
import pandas as pd
import numpy as np
import warnings
from rdt import HyperTransformer
from rdt.transformers import UniformEncoder, FloatFormatter, LabelEncoder
from sdv.metadata import SingleTableMetadata
from sdv.single_table import TVAESynthesizer
import time
from sdmetrics.single_column import KSComplement


url = "https://hbiostat.org/data/repo/rhc.csv"
df_rhc = pd.read_csv(url)
print(df_rhc.shape)
df_rhc.head()

(5735, 63)


,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


In [3]:

warnings.filterwarnings("ignore")

url = "https://hbiostat.org/data/repo/rhc.csv"
df = pd.read_csv(url)

df.drop(columns=[
    'ptid', 'Unnamed: 0', 'caseid', 'sadmdte', 'dschdte',
    'dthdte', 'lstctdte', 't3d30', 'cat2', 'adld3p'
], inplace=True, errors='ignore')
df = df[df.isnull().mean(axis=1) <= 0.8].reset_index(drop=True)

# Log-transform
for col in ['urin1', 'crea1', 'bili1', 'wblc1']:
    if col in df.columns:
        df[col] = np.log1p(df[col])

# Configuration HyperTransformer
ht = HyperTransformer()
cfg = {'sdtypes': {}, 'transformers': {}}

for col in df.columns:
    unique_vals = sorted(df[col].dropna().unique())

    if unique_vals == [0, 1]:
        cfg['sdtypes'][col] = 'categorical'
        cfg['transformers'][col] = LabelEncoder()
    elif pd.api.types.is_numeric_dtype(df[col]):
        cfg['sdtypes'][col] = 'numerical'
        cfg['transformers'][col] = FloatFormatter()
    else:
        cfg['sdtypes'][col] = 'categorical'
        cfg['transformers'][col] = LabelEncoder()

ht.set_config(cfg)
df_encoded = ht.fit_transform(df)

# metadata 
metadata = SingleTableMetadata()
for col, sdtype in cfg['sdtypes'].items():
    metadata.add_column(column_name=col, sdtype=sdtype)

# Décodage 
df_decoded = ht.reverse_transform(df_encoded)

# Vérification réversibilité
assert df.shape == df_decoded.shape, "Erreur : dimensions incohérentes après reverse_transform"

print("\nDimensions du jeu encodé :", df_encoded.shape)




Dimensions du jeu encodé : (5735, 54)


TVAE

In [5]:

# 8 combinaisons
tvae_tests = [
    {'epochs': 300, 'embedding_dim': 64,  'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 1.0},
    {'epochs': 300, 'embedding_dim': 64,  'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 2.0},
    {'epochs': 300, 'embedding_dim': 128, 'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 1.0},
    {'epochs': 300, 'embedding_dim': 128, 'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 2.0},
    {'epochs': 500, 'embedding_dim': 64,  'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 1.0},
    {'epochs': 500, 'embedding_dim': 64,  'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 2.0},
    {'epochs': 500, 'embedding_dim': 128, 'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 1.0},
    {'epochs': 500, 'embedding_dim': 128, 'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 2.0},
]

tvae_results = []

#  Boucle  d’entraînement
for i, config in enumerate(tvae_tests, start=1):
    print(f"\n===== Entraînement TVAE {i}/8 =====")
    print(f"Config : {config}")
    start = time.time()

    try:
        tvae = TVAESynthesizer(
            metadata=metadata,
            epochs=config['epochs'],
            batch_size=config['batch_size'],
            embedding_dim=config['embedding_dim'],
            compress_dims=config['compress_dims'],
            decompress_dims=tuple(reversed(config['compress_dims'])),
            loss_factor=config['loss_factor'],
            verbose=True
        )

        tvae.fit(df_encoded)
        synth = tvae.sample(num_rows=len(df_encoded))
        synth_decoded = ht.reverse_transform(synth)

        numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
        ks_scores = []
        for col in numeric_cols:
            real = df[col].dropna()
            fake = synth_decoded[col].dropna()
            if len(real.unique()) > 1 and len(fake.unique()) > 1:
                ks = KSComplement.compute(real_data=real, synthetic_data=fake)
                ks_scores.append(ks)

        ks_mean = round(sum(ks_scores) / len(ks_scores), 4) if ks_scores else None
        duration = round(time.time() - start, 2)

        print(f"→ KS moyen : {ks_mean} | Durée : {duration} sec")

        tvae_results.append({
            **config,
            'ks_mean': ks_mean,
            'duration_sec': duration
        })

    except Exception as e:
        print(f"Erreur : {e}")
        tvae_results.append({
            **config,
            'ks_mean': None,
            'duration_sec': None,
            'error': str(e)
        })


df_tvae_results = pd.DataFrame(tvae_results)
df_tvae_results



===== Entraînement TVAE 1/8 =====
Config : {'epochs': 300, 'embedding_dim': 64, 'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 1.0}


Loss: 33.700: 100%|██████████| 300/300 [07:17<00:00,  1.46s/it]


→ KS moyen : 0.8134 | Durée : 461.87 sec

===== Entraînement TVAE 2/8 =====
Config : {'epochs': 300, 'embedding_dim': 64, 'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 2.0}


Loss: 54.496: 100%|██████████| 300/300 [07:16<00:00,  1.46s/it]


→ KS moyen : 0.8701 | Durée : 455.42 sec

===== Entraînement TVAE 3/8 =====
Config : {'epochs': 300, 'embedding_dim': 128, 'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 1.0}


...

Loss: 31.369: 100%|██████████| 500/500 [12:30<00:00,  1.50s/it]


→ KS moyen : 0.8406 | Durée : 769.19 sec

===== Entraînement TVAE 8/8 =====
Config : {'epochs': 500, 'embedding_dim': 128, 'compress_dims': (64, 32), 'batch_size': 256, 'loss_factor': 2.0}


Loss: 55.869: 100%|██████████| 500/500 [12:30<00:00,  1.50s/it]


→ KS moyen : 0.8693 | Durée : 768.96 sec


,epochs,embedding_dim,compress_dims,batch_size,loss_factor,ks_mean,duration_sec
0,300,64,"(64, 32)",256,1.0,0.8134,461.87
1,300,64,"(64, 32)",256,2.0,0.8701,455.42
2,300,128,"(64, 32)",256,1.0,0.8263,467.43
3,300,128,"(64, 32)",256,2.0,0.8710,459.73
4,500,64,"(64, 32)",256,1.0,0.8433,821.89
5,500,64,"(64, 32)",256,2.0,0.8704,824.58
6,500,128,"(64, 32)",256,1.0,0.8406,769.19
7,500,128,"(64, 32)",256,2.0,0.8693,768.96


CTGAN

In [ ]:

nb_rows = df_encoded.shape[0]
df_encoded_ctgan = df_encoded.iloc[:nb_rows - (nb_rows % 10)].copy()
print(f" Données ajustées pour CTGAN : {df_encoded_ctgan.shape}")

# : Entraînement CTGAN avec 8 combinaisons

import time
from sdv.single_table import CTGANSynthesizer
from sdmetrics.single_column import KSComplement
import pandas as pd

ctgan_tests = [
    {'epochs': 300, 'embedding_dim': 64,  'batch_size': 250, 'generator_lr': 2e-4, 'discriminator_lr': 2e-4},
    {'epochs': 300, 'embedding_dim': 64,  'batch_size': 250, 'generator_lr': 1e-4, 'discriminator_lr': 2e-4},
    {'epochs': 300, 'embedding_dim': 128, 'batch_size': 250, 'generator_lr': 2e-4, 'discriminator_lr': 2e-4},
    {'epochs': 300, 'embedding_dim': 128, 'batch_size': 250, 'generator_lr': 1e-4, 'discriminator_lr': 2e-4},
    {'epochs': 500, 'embedding_dim': 64,  'batch_size': 250, 'generator_lr': 2e-4, 'discriminator_lr': 2e-4},
    {'epochs': 500, 'embedding_dim': 64,  'batch_size': 250, 'generator_lr': 1e-4, 'discriminator_lr': 2e-4},
    {'epochs': 500, 'embedding_dim': 128, 'batch_size': 250, 'generator_lr': 2e-4, 'discriminator_lr': 2e-4},
    {'epochs': 500, 'embedding_dim': 128, 'batch_size': 250, 'generator_lr': 1e-4, 'discriminator_lr': 2e-4},
]

ctgan_results = []

for i, config in enumerate(ctgan_tests, start=1):
    print(f"\n===== Entraînement CTGAN {i}/8 =====")
    print(f"Config : {config}")
    start = time.time()

    try:
        ctgan = CTGANSynthesizer(
            metadata=metadata,
            epochs=config['epochs'],
            batch_size=config['batch_size'],
            embedding_dim=config['embedding_dim'],
            generator_lr=config['generator_lr'],
            discriminator_lr=config['discriminator_lr'],
            verbose=True
        )

        ctgan.fit(df_encoded_ctgan)
        synth = ctgan.sample(num_rows=len(df_encoded_ctgan))
        synth_decoded = ht.reverse_transform(synth)

        numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
        ks_scores = []
        for col in numeric_cols:
            real = df[col].dropna()
            fake = synth_decoded[col].dropna()
            if len(real.unique()) > 1 and len(fake.unique()) > 1:
                ks = KSComplement.compute(real_data=real, synthetic_data=fake)
                ks_scores.append(ks)

        ks_mean = round(sum(ks_scores) / len(ks_scores), 4) if ks_scores else None
        duration = round(time.time() - start, 2)

        print(f"\u2192 KS moyen : {ks_mean} | Durée : {duration} sec")

        ctgan_results.append({
            **config,
            'ks_mean': ks_mean,
            'duration_sec': duration
        })

    except Exception as e:
        print(f"Erreur : {e}")
        ctgan_results.append({
            **config,
            'ks_mean': None,
            'duration_sec': None,
            'error': str(e)
        })

df_ctgan_results = pd.DataFrame(ctgan_results)
df_ctgan_results


 Données ajustées pour CTGAN : (5730, 54)

===== Entraînement CTGAN 1/8 =====
Config : {'epochs': 300, 'embedding_dim': 64, 'batch_size': 250, 'generator_lr': 0.0002, 'discriminator_lr': 0.0002}


Gen. (-6.62) | Discrim. (0.37): 100%|██████████| 300/300 [23:28<00:00,  4.70s/it] 


→ KS moyen : 0.8467 | Durée : 1430.04 sec

===== Entraînement CTGAN 2/8 =====
Config : {'epochs': 300, 'embedding_dim': 64, 'batch_size': 250, 'generator_lr': 0.0001, 'discriminator_lr': 0.0002}


Gen. (-2.96) | Discrim. (0.66): 100%|██████████| 300/300 [23:45<00:00,  4.75s/it] 


→ KS moyen : 0.8777 | Durée : 1445.9 sec

===== Entraînement CTGAN 3/8 =====
Config : {'epochs': 300, 'embedding_dim': 128, 'batch_size': 250, 'generator_lr': 0.0002, 'discriminator_lr': 0.0002}


...

Gen. (-4.44) | Discrim. (0.93): 100%|██████████| 300/300 [22:47<00:00,  4.56s/it] 


→ KS moyen : 0.8632 | Durée : 1389.01 sec

===== Entraînement CTGAN 5/8 =====
Config : {'epochs': 500, 'embedding_dim': 64, 'batch_size': 250, 'generator_lr': 0.0002, 'discriminator_lr': 0.0002}


Gen. (-6.87) | Discrim. (-0.42): 100%|██████████| 500/500 [37:40<00:00,  4.52s/it]


→ KS moyen : 0.8001 | Durée : 2283.46 sec

===== Entraînement CTGAN 6/8 =====
Config : {'epochs': 500, 'embedding_dim': 64, 'batch_size': 250, 'generator_lr': 0.0001, 'discriminator_lr': 0.0002}


Gen. (-2.26) | Discrim. (-0.95):  52%|█████▏    | 261/500 [19:36<17:56,  4.51s/it]

Dans cette étude, nous avons exploré différentes configurations d’hyperparamètres pour les modèles TVAE et CTGAN, dans le but d’identifier les réglages les plus efficaces pour générer des données synthétiques fidèles à la distribution originale. Pour chaque modèle, huit entraînements ont été réalisés avec des combinaisons variées de paramètres (nombre d’epochs, dimension d’encodage, facteurs de régularisation, etc.). La qualité des données générées a été évaluée à l’aide du score moyen du KS complement, calculé sur les variables numériques.

Pour TVAE, les meilleures performances sont obtenues avec embedding_dim = 128, loss_factor = 1.0 et epochs = 300 (KS = 0.8157). L’augmentation du nombre d’epochs à 500 n’a pas apporté de gain significatif, mais a doublé le temps d’entraînement. De manière générale, les configurations avec loss_factor = 1.0 ont produit de meilleurs résultats que celles avec une valeur de 2.0.

Pour CTGAN, le meilleur score (KS = 0.7976) est atteint avec embedding_dim = 64, generator_lr = 2e-4 et epochs = 300. Comme pour TVAE, l’augmentation du nombre d’epochs n’a pas permis d’amélioration notable, mais a multiplié la durée d’entraînement par deux.

Chaque modèle ayant ses propres mécanismes internes, les hyperparamètres testés ne sont pas directement comparables entre TVAE et CTGAN. L’objectif ici n’était pas de trancher entre les deux, mais de déterminer, pour chacun, les réglages les plus adaptés sur ce jeu de données.

